REMEMBER: FIRST CREATE A COPY OF THIS FILE WITH A UNIQUE NAME AND DO YOUR WORK THERE. AND MAKE SURE YOU COMMIT YOUR CHANGES TO THE `hw3_submissions` BRANCH.

# Assignment 3 | Cleaning and Exploring Data with Pandas



<img src="data/scoreCard.jpg" width=250>

In this assignment, you will investigate restaurant food safety scores for restaurants in San Francisco. Above is a sample score card for a restaurant. The scores and violation information have been made available by the San Francisco Department of Public Health. 

## Loading Food Safety Data


There are 2 files in the data directory:
1. business.csv containing food establishments in San Francisco
1. inspections.csv containing retaurant inspections records

Let's start by loading them into Pandas dataframes.  One of the files, business.csv, has encoding (ISO-8859-1), so you will need to account for that when reading it.

### Question 1

#### Question 1a
Read the two files noted above into two pandas dataframes named `bus` and `ins`, respectively. Print the first 5 rows of each to inspect them.


In [1]:
import pandas as pd

In [2]:
bus = pd.read_csv('data/businesses.csv', encoding='ISO-8859-1')
ins = pd.read_csv('data/inspections.csv')

In [3]:
bus.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494
3,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN
4,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051


In [4]:
ins.head()

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine


## Examining the Business data

From its name alone, we expect the `businesses.csv` file to contain information about the restaurants. Let's investigate this dataset.

### Question 2

#### Question 2a: How many records are there?

In [5]:
bus.shape

(6315, 9)

Ans: There are 6315 records.

#### Question 2b: How many unique business IDs are there?  

In [6]:
num_unique_business_ids = bus['business_id'].nunique()
print(num_unique_business_ids)

6315


Ans: There are 6315 unique business IDs.

#### Question 2c: What are the 5 most common businesses by name, and how many are there in San Francisco?

In [7]:
business_counts = bus['name'].value_counts()
top_5_businesses = business_counts.head(5)
print(top_5_businesses)


name
STARBUCKS COFFEE              72
PEET'S COFFEE & TEA           24
MCDONALDS                     12
SAN FRANCISCO SOUP COMPANY    11
WALGREENS                     11
Name: count, dtype: int64


In [8]:
sf_businesses = bus[bus['city'] == 'San Francisco']
num_top_5_businesses_in_sf = sf_businesses[sf_businesses['name'].isin(top_5_businesses.index)].shape[0]
print("Number of the top 5 businesses in San Francisco:", num_top_5_businesses_in_sf)

Number of the top 5 businesses in San Francisco: 130


Ans: 
1. 5 most common businesses by name:    
   STARBUCKS COFFEE, PEET'S COFFEE & TEA, MCDONALDS, SAN FRANCISCO SOUP COMPANY, WALGREENS
2. 130 stores of the 5 most common businesses are in San Francisco.

## Zip code

Next, let's  explore some of the variables in the business table. We begin by examining the postal code.

### Question 3

#### Question 3a
How are the zip code values stored in python (i.e. data type)?

To answer this you might want to examine a particular entry.

In [9]:
zip_code_type = bus.loc[0, 'postal_code']
print("Data type of the zip code values in python: ", type(zip_code_type))

Data type of the zip code values in python:  <class 'str'>


Ans: Zip code values stored in python as "str" (string).

#### Question 3b

What are the unique values of postal_code?

In [10]:
unique_postal_code = bus['postal_code'].unique()
print("Unique values of postal_code: ")
print(unique_postal_code)

Unique values of postal_code: 
['94104' '94109' '94133' '94110' '94122' '94103' '94115' '94131' '94111'
 '94117' '94107' '94108' '94102' '94132' '94105' '94134' '94124' '94116'
 '94121' '94118' '94112' '94123' '94114' '94127' '941102019' '941' '94545'
 '94066' '941033148' nan '94158' '95105' '94013' '94130' 'CA' '92672'
 '94120' '94143' '94609' '94101' '00000' '94188' '94621' '94014' '94129'
 '94602' 'Ca']


#### Question 3c

Let's say we decide to exclude the businesses that have no zipcode for our analysis (which might include food trucks for example).  Use the list of valid 5-digit zip codes below to create a new dataframe called bus_valid, with only businesses whose postal_codes show up in this list of valid zipcodes. How many businesses are there in this new dataframe?

In [11]:
validZip = ["94102", "94103", "94104", "94105", "94107", "94108",
            "94109", "94110", "94111", "94112", "94114", "94115",
            "94116", "94117", "94118", "94121", "94122", "94123", 
            "94124", "94127", "94131", "94132", "94133", "94134"]

In [12]:
vb = bus[bus['postal_code'].isin(validZip)]

In [13]:
num_bus_valid = len(vb)
print("Number of businesses in the bus_valid dataframe (with valid 5-digit zip): ", num_bus_valid)

Number of businesses in the bus_valid dataframe (with valid 5-digit zip):  5999


Ans: 5999 businesses are in this new dataframe.

## Latitude and Longitude

Another aspect of the data we want to consider is the prevalence of missing values. If many records have missing values then we might be concerned about whether the nonmissing values are representative of the population.

### Question 4
 
Consider the longitude and latitude in the business DataFrame. 

#### Question 4a

How many businesses are missing longitude values, working with only the businesses that are in the list of valid zipcodes?

In [14]:
vb[pd.isnull(vb['longitude'])]

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
200,821,LA SANTANECA,3781 MISSION ST,San Francisco,CA,94110,NaN,NaN,+14156481034
210,852,NEW HOLLYWOOD BAKERY & RESTAURANT,652 PACIFIC AVE,San Francisco,CA,94133,NaN,NaN,+14153979919
760,2934,LOU'S FISH SHACK,300 JEFFERSON ST,San Francisco,CA,94133,NaN,NaN,+14157715687
826,3123,LES JOULINS,44 ELLIS ST,San Francisco,CA,94102,NaN,NaN,+14153975397
1100,4323,DIVA'S,1081 POST ST,San Francisco,CA,94109,NaN,NaN,+14154743482
...,...,...,...,...,...,...,...,...,...
6310,90234,ACCESS SFUSD THE ARC,1520 HOWARD ST,San Francisco,CA,94103,NaN,NaN,NaN
6311,90236,PHO DONG HUONG,667 MONTEREY BLVD,San Francisco,CA,94127,NaN,NaN,+14154945048
6312,90237,WIN'S RESTAURANT,3040 TARAVAL ST,San Francisco,CA,94116,NaN,NaN,+14158606036
6313,90238,USA POWER MARKET,1524 OCEAN AVE,San Francisco,CA,94112,NaN,NaN,+14153331878


Ans: 2483 businesses are missing longitude values.

#### Question 4b

Create a new dataframe with one row for each valid zipcode. The dataframe should include the following three columns:

1. `postal_code`: Contains the zip codes in the `validZip` variable above.
2. `null_lon`: The number of businesses in that zipcode with missing `longitude` values.
3. `not_null_lon`: The number of businesses without missing `longitude` values.

In [15]:
zipcode_stats = vb.groupby('postal_code')['longitude'].agg(null_lon=lambda x: sum(pd.isnull(x)), not_null_lon=lambda x: sum(pd.notnull(x))).reset_index()
zipcode_stats.columns = ['postal_code', 'null_lon', 'not_null_lon']

zipcode_stats

,postal_code,null_lon,not_null_lon
0,94102,196,262
1,94103,252,306
2,94104,59,74
3,94105,100,126
4,94107,247,204
5,94108,82,142
6,94109,146,237
7,94110,251,334
8,94111,115,167
9,94112,59,131


#### 4c. Do any zip codes appear to have more than their 'fair share' of missing longitude? 

To answer this, you will want to compute the proportion of missing longitude values for each zip code, and print the proportion missing longitude, and print the top five zipcodes in descending order of proportion missing postal_code.


In [16]:
zipcode_stats['prop_missing_longitude (%)'] = (zipcode_stats['null_lon'] / (zipcode_stats['null_lon'] + zipcode_stats['not_null_lon'])) * 100
sorted_zipcodes = zipcode_stats.sort_values(by='prop_missing_longitude (%)', ascending=False)
top_5_zipcodes = sorted_zipcodes.head(5)
top_5_zipcodes

,postal_code,null_lon,not_null_lon,prop_missing_longitude (%)
4,94107,247,204,54.767184
18,94124,88,76,53.658537
21,94132,65,69,48.507463
10,94114,108,122,46.956522
16,94122,123,147,45.555556


Ans: Zip codes 94107, 94124, 94132, 94114, and 94122 appear to have more than their 'fair share' of missing longitude.

# Investigate the inspection data

Let's now turn to the inspection DataFrame. Earlier, we found that `ins` has 4 columns, these are named `business_id`, `score`, `date` and `type`.  In this section, we determine the granularity of `ins` and investigate the kinds of information provided for the inspections. 

### Question 5

#### Question 5a
As with the business data, assess whether there is one inspection record for each business, by counting how many rows are in the data and how many unique businesses there are in the data. If they are exactly the same number, it means there is only one inspection per business, clearly.

In [17]:
print(num_unique_business_ids)

6315


In [18]:
ins.shape

(15430, 4)

Ans: The inspection data has more records than the number of unique business ids (15430 > 6315). There is not is only one inspection per business.

#### Question 5b

What values does `type` take on? How many occurrences of each value is in the DataFrame? Create a new dataframe named `ins2` by copying `ins` and keeping only records with values of `type` that occur more than 10 times in the original table. In other words, eliminate records that have values of `type` that occur rarely (< 10 times). Check the result to make sure rare types are eliminated.

In [19]:
ins.head()

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine


In [20]:
ins.dtypes

business_id     int64
score           int64
date            int64
type           object
dtype: object

In [21]:
from pandas.api.types import is_string_dtype

is_string_dtype(ins['type'])

True

In [22]:
type_counts = ins['type'].value_counts()
print("Unique string values of 'type' and their occurrences:")
print(type_counts)

Unique string values of 'type' and their occurrences:
type
routine      15429
complaint        1
Name: count, dtype: int64


In [23]:
complaint_df = ins[ins['type'] == 'complaint']
complaint_df

,business_id,score,date,type
15291,87440,100,20160801,complaint


In [24]:
ins2 = ins.copy()
ins2 = ins2.groupby('type').filter(lambda x: len(x) > 10)

type_counts_ins2 = ins2['type'].value_counts()
print(type_counts_ins2)

type
routine    15429
Name: count, dtype: int64


Ans: "type" takes on strings. "routine" appears 15429 times and "complaint" appears "1" time.

#### Question 5c

Since the data was stored in a .csv file, the dates are formatted as strings such as `20160503`. Once we read in the data, we would like to have dates in an appropriate format for analysis. Add a new column called `year` by capturing the first four characters of the date column. 

Hint: we have seen multiple ways of doing this in class, includings `str` operations, `lambda` functions, `datetime` operations, and others. Choose the method that works best for you :)

In [25]:
ins2['date'] = ins2['date'].astype(str)
ins2['year'] = ins2['date'].str[:4].astype(int)
ins2

,business_id,score,date,type,year
0,10,82,20160503,routine,2016
1,10,94,20140729,routine,2014
2,10,92,20140114,routine,2014
3,19,94,20160513,routine,2016
4,19,94,20141110,routine,2014
...,...,...,...,...,...
15425,89515,98,20161103,routine,2016
15426,89701,100,20161206,routine,2016
15427,89790,96,20161129,routine,2016
15428,89803,100,20161215,routine,2016


#### Question 5d

What range of years is covered in this data set? Are there roughly same number of inspections each year? Try dropping records for any years with less than 50 inspections and store the result in a new dataframe named `ins3`.

In [26]:
year_range = sorted(ins2['year'].unique())
print("Range of years covered in the dataset:", year_range)

Range of years covered in the dataset: [2013, 2014, 2015, 2016]


In [27]:
inspection_counts_ins2 = ins2['year'].value_counts()
print("Number of inspections each year:")
print(inspection_counts_ins2)

Number of inspections each year:
year
2016    5839
2014    5629
2015    3923
2013      38
Name: count, dtype: int64


In [28]:
ins3 = ins2.groupby('year').filter(lambda x: len(x) >= 50)
inspection_counts_ins3 = ins3['year'].value_counts()
print(inspection_counts_ins3)

year
2016    5839
2014    5629
2015    3923
Name: count, dtype: int64


Let's examine only the inspections for one year: 2016. This puts businesses on a more equal footing because [inspection guidelines](https://www.sfdph.org/dph/eh/Food/Inspections.asp) generally refer to how many inspections should occur in a given year.

In [29]:
ins_2016 = ins3[ins3['year'] == 2016]
ins_2016

,business_id,score,date,type,year
0,10,82,20160503,routine,2016
3,19,94,20160513,routine,2016
6,24,98,20161005,routine,2016
7,24,96,20160311,routine,2016
10,31,84,20160816,routine,2016
...,...,...,...,...,...
15425,89515,98,20161103,routine,2016
15426,89701,100,20161206,routine,2016
15427,89790,96,20161129,routine,2016
15428,89803,100,20161215,routine,2016


### Question 6

#### Question 6a

Merge the business and 2016 inspections data, keeping all businesses regardless of whether they show up in the inspections file. Show the first several rows of the resulting dataframe.

In [30]:
merged_data_2016 = pd.merge(vb, ins_2016, on='business_id', how='left')
merged_data_2016['year'] = merged_data_2016['year'].fillna(0)
merged_data_2016['year'] = merged_data_2016['year'].astype(int)

merged_data_2016.head(10)

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number,score,date,type,year
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044,82.0,20160503,routine,2016
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262,94.0,20160513,routine,2016
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,98.0,20161005,routine,2016
3,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,96.0,20160311,routine,2016
4,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN,84.0,20160816,routine,2016
5,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051,84.0,20160614,routine,2016
6,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051,78.0,20160104,routine,2016
7,48,ART'S CAFE,747 IRVING ST,San Francisco,CA,94122,37.764013,-122.465749,+14156657440,94.0,20160630,routine,2016
8,50,SUSHI ZONE,1815 MARKET ST.,San Francisco,CA,94103,37.771437,-122.423892,+14156211114,80.0,20160614,routine,2016
9,54,RHODA GOLDMAN PLAZA,2180 POST ST,San Francisco,CA,94115,37.784626,-122.437734,+14153455060,NaN,NaN,NaN,0


In [31]:
merged_data_2016.shape

(7270, 13)

#### Question 6b
Print the 20 lowest rated businesses names, their addresses, and their ratings.

In [32]:
lowest_rated_businesses = merged_data_2016.sort_values(by='score', ascending=True)
lowest_rated_businesses = lowest_rated_businesses.head(20)

print("20 lowest rated businesses:")
print(lowest_rated_businesses[['name', 'address', 'score']])

20 lowest rated businesses:
                             name              address  score
4755      GOLDEN RIVER RESTAURANT     5827 GEARY BLVD    52.0
4736                   UNCLE CAFE       65 WAVERLY PL    55.0
6042                 CRAZY PEPPER   2257 SAN JOSE AVE    55.0
7020                    POKI TIME     2101 LOMBARD ST    55.0
5448                   GOLDEN WOK   295 B ORIZABA AVE    56.0
2968             HING WANG BAKERY        339 JUDAH ST    56.0
836                  RED A BAKERY      634 CLEMENT ST    57.0
5861    L & G VIETNAMESE SANDWICH         602 EDDY ST    57.0
4107      SAN TUNG RESTAURANT LLC      1031 IRVING ST    57.0
4766  NEW GARDEN RESTAURANT, INC.       716 KEARNY ST    57.0
549           MA'S DIM SUM & CAFE      1315 POWELL ST    57.0
5214                   TOP'S CAFE  2460 SAN BRUNO AVE    57.0
6386    KOBANI MEDITERANEAN GRILL       3242 SCOTT ST    58.0
4130                        ITHAI         720 POST ST    59.0
624           JOANIE'S DINER INC.   1329 C

## Done!

Now commit this notebook to your `hw3_submissions` branch, push it to your GitHub repo, and open a PR!